# **Fantasy Model Application and Database**

---

This application aims to keep a historical record of any fantasy football league and provide some visualizations for league data.
<br><br>
***HOW TO RUN THIS APPLICATION:***
- First, run the **Build** section. This sets up global functions and the application entities.
- Then, run the **Create** section. This creates the application and adds data to the database.
- Finally, run any of the cells in the **Run** section in any order.

## Build

---

You can collapse this section and run all cells to set up the application and database.

### Imports

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import decimal
from IPython.display import display

### Classes

#### Application

In [2]:
class Application():
  def __init__(self):
    self.db = Database()
    self.visualizer = Visualizer()

  def displayLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df)
  
  def displayLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df, self.db.manager_record_df)

  def displayActiveLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0])
  
  def displayActiveLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df)

  def displayManagerRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df, self.db.manager_record_df, sort_key)

  def displayActiveManagerRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df, sort_key)

  def displayRecordsForManager(self, manager):
    # display(self.db.manager_record_df.loc[self.db.manager_record_df['abbreviation'] == manager].style.hide_index())
    self.visualizer.createRecordsForManagerDisplay(self.db.manager_df, self.db.manager_record_df, manager)

  def displayTotalChampionshipsByManager(self):
    self.visualizer.createTotalChampionshipsDisplay(pd.merge(self.db.manager_df, self.db.top_three_finish_df, on='abbreviation'))

  def displayTopThreeFinishes(self):
    self.visualizer.createTopThreeFinishesDisplay(self.db.manager_df, self.db.top_three_finish_df)

#### Database

In [3]:
class Database():
  def __init__(self):
    self.manager_df = ManagerHelper.createManagerDF()
    self.manager_record_df = RecordHelper.createManagerRecordDF()
    self.game_df = GameHelper.createGameDF()
    self.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()
    self.playoff_game_df= PlayoffHelper().createPlayoffGameDF()

  def addManagers(self, managers):
    self.manager_df = ManagerHelper.addManagers(self.manager_df, managers)

  def addActiveYears(self):
    self.manager_df = ManagerHelper.addActiveYears(self.manager_df, self.manager_record_df)

  def addGamesForYearAndWeek(self, year, week, games):
    self.game_df = GameHelper.addGamesForYearAndWeek(self.game_df, year, week, games)

  def addLegacyRecordsForYear(self, year, records):
    self.manager_record_df = RecordHelper.addLegacyRecordsForYear(self.manager_record_df, year, records)

  def addPlayoffGamesForYearAndRound(self, year, game_round, playoff_games):
    self.playoff_game_df = PlayoffHelper.addPlayoffGamesForYearAndRound(self.playoff_game_df, year, game_round, playoff_games)

  def addTopThreeFinishesForYear(self, year, finishes):
    self.top_three_finish_df = TopThreeFinishHelper.addTopThreeFinishesForYear(self.top_three_finish_df, year, finishes)

  def addRecordsFromGames(self):
    self.manager_record_df = RecordHelper.addRecordsFromGames(self.game_df, self.manager_record_df)

##### Manager Helper

In [4]:
class ManagerHelper():
  # Create Manager DataFrame
  def createManagerDF():
    df = pd.DataFrame()
    df['name'] = []
    df['abbreviation'] = []
    df['active'] = []
    df = df.astype({
        'active': int })
    return df

  def addManager(name, abbreviation, active, manager_df):
    dict = {
        'name': name,
        'abbreviation': abbreviation,
        'active': active
    }
    manager_df = manager_df.append(dict, ignore_index = True)
    return manager_df

  def addManagers(manager_df, managers):
    for manager in managers:
      name = manager[0]
      abbreviation = manager[1]
      active = manager[2]
      manager_df = ManagerHelper.addManager(name, abbreviation, active, manager_df)
    return manager_df

  def addActiveYears(manager_df, manager_record_df):
    return_df = pd.DataFrame()
    return_df['abbreviation'] = []
    return_df['active_years'] = []
    manager_list = manager_df['abbreviation']
    for manager in manager_list:
      result_df = manager_record_df.loc[manager_record_df['abbreviation'] == manager]
      active_years = ""
      for index, row in result_df.iterrows():
        year = row['season_start_year']
        active_years += str(year) + ", "
      active_years = active_years[0:-2]
      dict = {
          'abbreviation': manager,
          'active_years': active_years
      }
      return_df = return_df.append(dict, ignore_index=True)
    df = pd.merge(manager_df, return_df, on='abbreviation')
    return df

##### Record Helper

In [5]:
class RecordHelper():
  # Create League Manager Records DataFrame
  def createManagerRecordDF():
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['wins'] = []
    df['losses'] = []
    df['ties'] = []
    df['pf'] = []
    df['pa'] = []
    df = df.astype({
        'season_start_year': int,
        'wins': int,
        'losses': int,
        'ties': int })
    return df

  def addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': 0
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': ties
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordsForYear(manager_record_df, season_start_year, records):
    for record in records:
      manager = record[0]
      wins = record[1]
      losses = record[2]
      if len(record) == 4:
        ties = record[3]
        manager_record_df = RecordHelper.addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties)
        continue
      manager_record_df = RecordHelper.addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses)
    return manager_record_df

  def addRecordsFromGames(game_df, manager_record_df):
    calculated_df = pd.DataFrame()
    calculated_df['abbreviation'] = pd.Series(dtype='object')
    calculated_df['season_start_year'] = pd.Series(dtype=int)
    calculated_df['wins'] = pd.Series(dtype=int)
    calculated_df['losses'] = pd.Series(dtype=int)
    calculated_df['ties'] = pd.Series(dtype=int)
    calculated_df['pf'] = pd.Series(dtype=float)
    calculated_df['pa'] = pd.Series(dtype=float)
    years_in_game_df = game_df['season_start_year'].unique()
    for year in years_in_game_df:
      year_game_df = game_df.loc[game_df['season_start_year'] == year]
      manager_list1 = np.array(year_game_df['manager1'].unique())
      manager_list2 = np.array(year_game_df['manager2'].unique())
      manager_list = np.concatenate((manager_list1, manager_list2))
      manager_list = np.unique(manager_list)
      for manager in manager_list:
        manager_games1 = year_game_df.loc[year_game_df['manager1'] == manager]
        manager_games2 = year_game_df.loc[year_game_df['manager2'] == manager]
        df = pd.concat([manager_games1, manager_games2])
        wins = 0
        losses = 0
        ties = 0
        pf = 0
        pa = 0
        for index, row in df.iterrows():
          manager1_score = row['manager1_score']
          manager2_score = row['manager2_score']
          if (row['manager1'] == manager):
            if (manager1_score > manager2_score):
              wins += 1
            elif (manager1_score < manager2_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager1_score
            pa += manager2_score
          else:
            if (manager2_score > manager1_score):
              wins += 1
            elif (manager2_score < manager1_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager2_score
            pa += manager1_score
        dict = {
            'abbreviation': manager,
            'season_start_year': year,
            'wins': wins,
            'losses': losses,
            'ties': ties,
            'pf': pf,
            'pa': pa
        }
        calculated_df = calculated_df.append(dict, ignore_index = True)
    df = pd.concat([manager_record_df, calculated_df])
    return df

  def determine_tie(row, manager):
    return 0

##### Game Helper

In [6]:
class GameHelper():
  # Create Regular Season Games DataFrame
  def createGameDF():
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_score'] = []
    df['season_start_year'] = []
    df['week'] = []
    df = df.astype({
        'season_start_year': int, 
        'week': int })
    return df

  def addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week):
    dict = {
        'manager1': manager1, 
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_score': manager2_score,
        'season_start_year': season_start_year,
        'week': week
    }
    game_df = game_df.append(dict, ignore_index = True)
    return game_df

  def addGamesForYearAndWeek(game_df, year, week, games):
    for game in games:
      manager1 = game[0]
      manager1_score = game[1]
      manager2 = game[2]
      manager2_score = game[3]
      season_start_year = year
      week = week
      game_df = GameHelper.addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week)
    return game_df

##### Top Three Finish Helper

In [7]:
# TODO - make this top four

class TopThreeFinishHelper():
  # Create League Manager Top Three Finishes DataFrame
  def createTopThreeFinishesDF(self):
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['finishing_position'] = []
    df = df.astype({
        'season_start_year': int,
        'finishing_position': int })
    return df

  def addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'finishing_position': finishing_position
    }
    top_three_finish_df = top_three_finish_df.append(dict, ignore_index=True)
    return top_three_finish_df

  def addTopThreeFinishesForYear(top_three_finish_df, season_start_year, finishes):
    for finish in finishes:
      manager = finish[0]
      finishing_position = finish[1]
      top_three_finish_df = TopThreeFinishHelper.addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position)
    return top_three_finish_df

##### Playoff Helper

In [8]:
class  PlayoffHelper():
  # Create Playoff Game DataFrame
  def createPlayoffGameDF(self):
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_rank'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_rank'] = []
    df['manager2_score'] = []
    df['round'] = []
    df['season_start_year'] = []
    df = df.astype({
        'season_start_year': int, 
        'round': int,
        'manager1_rank': int,
        'manager2_rank': int })
    return df

  def addPlayoffGamesForYearAndRound(playoff_game_df, year, round, playoff_games):
    for game in playoff_games:
      manager1 = game[0]
      manager1_rank = game[1]
      manager1_score = game[2]
      manager2 = game[3]
      manager2_rank = game[4]
      manager2_score = game[5]
      playoff_game_df = PlayoffHelper.addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year)
    return playoff_game_df

  def addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year):
    dict = {
        'manager1': manager1, 
        'manager1_rank': manager1_rank,
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_rank': manager2_rank,
        'manager2_score': manager2_score,
        'round': round,
        'season_start_year': year
    }
    playoff_game_df = playoff_game_df.append(dict, ignore_index = True)
    return playoff_game_df

##### Sorting Helper

In [9]:
class sort_by():
  # Define Sort Keys
  name = 'name'
  wins = 'wins'
  losses = 'losses'
  ties = 'ties'
  win_pct = 'win_pct'

#### Visualizer

In [10]:
class Visualizer():
  def __init__(self):
    pd.set_option('max_colwidth', None)
    pd.set_option('colheader_justify', 'center')

  def createLeagueManagersDisplay(self, df):
    df = df.sort_values(by=['name'])
    df = df[['name', 'active_years']]
    df = df.rename(columns={
        'name':'Name', 
        'active_years': 'Active Years' })
    df = df.style.hide_index()
    display(df)

  def createLeagueManagersDisplay2(self, manager_df, manager_record_df):
    # select name and active years column sorted by name
    manager_df = manager_df[['name', 'active_years']]
    manager_df = manager_df.sort_values(by=['name'])

    # create empty display df and dict with name and years
    display_df = pd.DataFrame()
    display_df['name'] = []
    dict = {'name':''}
    years = manager_record_df['season_start_year'].unique()
    for year in years:
      display_df[year] = []
      dict[year] = ''

    # add green bar for active years
    managers_list = manager_df['name']
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      manager_active_years = manager_df.loc[manager_df['name'] == manager]['active_years']
      for year in years:
        if str(year) in str(manager_active_years):
          dict[year] = "🟩🟩🟩"
        else:
          dict[year] = ""
      display_df = display_df.append(dict, ignore_index=True)

    # clean up and display
    display_df = display_df.rename(columns={'name':'Name'})
    display_df = display_df.style.hide_index()
    display(display_df)

  def createManagerRecordDisplay(self, manager_df, manager_record_df, sort_key):
    # join manager_df and manager_record_df
    df = pd.merge(manager_df, manager_record_df, on='abbreviation')
    # get total wins, losses, and ties
    df = df.groupby('name', as_index=False).sum()
    # select name, wins, losses, and ties
    df = df[['name', 'wins', 'losses', 'ties']]

    # calculate win percentsge
    df['win_pct'] = pd.Series(dtype='int')
    for index, row in df.iterrows():
      wins = row['wins']
      losses = row ['losses']
      ties = row['ties']
      total_games = wins + losses + ties
      win_percent = wins / total_games
      c = decimal.Decimal(win_percent * 100)
      df.at[index, 'win_pct'] = float(round(c, 3))

    # sort by sort_key
    df = df.sort_values(by=[sort_key], ascending=False)

    # clean up and display
    df = df.rename(columns={
        'name':'Name', 
        'wins': 'Wins',
        'losses': 'Losses',
        'ties': 'Ties',
        'win_pct': 'Win Pct' })
    df = df.style
    df = df.format(precision=3)
    df = df.hide_index()
    display(df)

  def createTotalChampionshipsDisplay(self, df):
    df = df.loc[df['finishing_position'] == 1]
    df = df.groupby('name', as_index=False).sum()
    df = df.sort_values(by=['finishing_position'], ascending=False)
    df = df[['name', 'finishing_position']]
    df = df.rename(columns={'name':'Name','finishing_position': 'Championships'})
    df = df.style.hide_index()
    display(df)

  def createTopThreeFinishesDisplay(self, manager_df, top_three_finish_df):
    df = pd.merge(manager_df, top_three_finish_df, on='abbreviation')
    df2 = df[['name', 'season_start_year', 'finishing_position']]
    display_df = pd.DataFrame()
    display_df['name'] = []
    years_in_top_3_finishes = top_three_finish_df['season_start_year'].unique()
    dict = {'name':''}
    for year in years_in_top_3_finishes:
      display_df[year] = []
      dict[year] = ''
    dict['total'] = []
    managers_list = df2['name'].unique()
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      top_three_finishes = df2.loc[df2['name'] == manager][['season_start_year', 'finishing_position']]
      total_top_three = 0
      for year in years_in_top_3_finishes:
        if (top_three_finishes['season_start_year'].eq(year)).any():
          finishing_position = df[(df['season_start_year'] == year) & (df['name'] == manager)]
          finishing_position = finishing_position['finishing_position'].item()
          if (finishing_position == 3):
            dict[year] = "🥉"
          elif (finishing_position == 2):
            dict[year] = "🥈"
          else:
            dict[year] = "🥇"
          total_top_three += 1
        else:
          dict[year] = ""
        dict['total'] = total_top_three
      display_df = display_df.append(dict, ignore_index=True)
    display_df = display_df.astype({'total': int})
    display_df = display_df.sort_values(by=['total'], ascending=False)
    display_df = display_df.rename(columns={'name':'Name','total': 'Total'})
    display_df = display_df.style.hide_index()
    pd.set_option('colheader_justify', 'left')
    display(display_df)

  def createRecordsForManagerDisplay(self, manager_df, manager_record_df, manager):
    df = pd.merge(manager_df, manager_record_df, on='abbreviation')
    df = df.loc[df['name'] == manager]
    df = df[['name', 'season_start_year',	'wins',	'losses',	'ties',	'pf',	'pa']]
    df = df.fillna("-")
    df = df.rename(columns={
        'name':'Name',
        'season_start_year': 'Year',
        'wins': "Wins",
        'losses': 'Losses',
        'ties': 'Ties',
        'pf': 'PF',
        'pa': 'PA' })
    df = df.sort_values(by=['Year'])
    df = df.style
    df = df.set_properties(**{'text-align': 'center'})
    df = df.format(precision=2)
    df = df.hide_index()
    df.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df)

## Create

---

You can collapse this section and run all cells to create the application and add data to the database.

### Initialize

In [17]:
# Initilaize Application
app = Application()

### Add Managers

In [18]:
# Add Managers

# Reset Managers DataFrame
app.db.manager_df = ManagerHelper.createManagerDF()

# Create Mananger Abbreviations
MGR1 = '0001'
MGR2 = '0002'
# ...
# ...
# ...
MGRM = '000M'
MGRN = '000N'

# Add Manager Name, Abbreviation, and Active Status to DB
app.db.addManagers([
    ['Manager 1 Name', MGR1, 1],
    ['Manager 2 Name', MGR2, 1],
    # ...
    # ...
    # ...
    ['Manager N-1 Name', MGRM, 1],
    ['Manager N Name', MGRN, 1]
])

### Add Records

In [19]:
# Add Legacy Records

# Reset Records DB
app.db.manager_record_df = RecordHelper.createManagerRecordDF()

# Add Legacy Records (2020)
app.db.addLegacyRecordsForYear(2020, [
    [MGR1, 7, 6],
    [MGR2, 9, 4],
    # ...
    # ...
    # ...
    [MGRM, 7, 6],
    [MGRN, 10, 3]
])

# Add Legacy Records (2021)
app.db.addLegacyRecordsForYear(2021, [
    [MGR1, 11, 3],
    [MGR2, 10, 4],
    # ...
    # ...
    # ...
    [MGRM, 6, 8],
    [MGRN, 7, 7]
])

### Add Games

In [20]:
# Reset Games DF
app.db.game_df = GameHelper.createGameDF()

#### 2022

In [21]:
# Add Regular Season Games for a Year

# Week 1
app.db.addGamesForYearAndWeek(2022, 1, [
    [MGR1, 77.25, MGR2, 129.9],
    # ...
    [MGRM, 120.65, MGRN, 115.35]
])

# Week 2
app.db.addGamesForYearAndWeek(2022, 2, [
    [MGR1, 116.9, MGR2, 140.8],
    # ...
    [MGRM, 114.85, MGRN, 128.75]
])

# Week 3
app.db.addGamesForYearAndWeek(2022, 3, [
    [MGR1, 111.3, MGR2, 76.5],
    # ...
    [MGRM, 131.9, MGRN, 76.35]
])

# Week 4
app.db.addGamesForYearAndWeek(2022, 4, [
    [MGR1, 134.55, MGR2, 124.15],
    # ...
    [MGRM, 101.55, MGRN, 121.15]
])

# Week 5
app.db.addGamesForYearAndWeek(2022, 5, [
    [MGR1, 104.4, MGR2, 130.2],
    # ...
    [MGRM, 144.4, MGRN, 102.15]
])

# Week 6
app.db.addGamesForYearAndWeek(2022, 6, [
    [MGR1, 96.65, MGR2, 84.15],
    # ...
    [MGRM, 122.0, MGRN, 89.8]
])

# Week 7
app.db.addGamesForYearAndWeek(2022, 7, [
    [MGR1, 105.9, MGR2, 88.95],
    # ...
    [MGRM, 90.0, MGRN, 57.6]
])

# Week 8
app.db.addGamesForYearAndWeek(2022, 8, [
    [MGR1, 98.05, MGR2, 125.7],
    # ...
    [MGRM, 137.9, MGRN, 156.4]
])

# Week 9
app.db.addGamesForYearAndWeek(2022, 9, [
    [MGR1, 111.05, MGR2, 92.45],
    # ...
    [MGRM, 104.9, MGRN, 110.05]
])

# Week 10
app.db.addGamesForYearAndWeek(2022, 10, [
    [MGR1, 96.15, MGR2, 115.95],
    # ...
    [MGRM, 147.65, MGRN, 119.7]
])

# Week 11
app.db.addGamesForYearAndWeek(2022, 11, [
    [MGR1, 110.6, MGR2, 115.65],
    # ...
    [MGRM, 89.95, MGRN, 95.55]
])

# Week 12
app.db.addGamesForYearAndWeek(2022, 12, [
    [MGR1, 122.1, MGR2, 82.0],
    # ...
    [MGRM, 102.2, MGRN, 120.05]
])

# Week 13
app.db.addGamesForYearAndWeek(2022, 13, [
    [MGR1, 122.85, MGR2, 98.25],
    # ...
    [MGRM, 78.25, MGRN, 127.1]
])

# Week 14
app.db.addGamesForYearAndWeek(2022, 14, [
    [MGR1, 146.15, MGR2, 89.05],
    # ...
    [MGRM, 149.7, MGRN, 55.65]
])

In [22]:
# Add Playoff Games for ____

# Round 1
app.db.addPlayoffGamesForYearAndRound(2022, 1, [
    [MGR1, 1, 129.4, MGR2, 4, 155.6],
    # ...
    [MGRM, 3, 75.7, MGRN, 2, 113.3]
])

# Round 2
app.db.addPlayoffGamesForYearAndRound(2022, 2, [
    [MGR2, 4, 79.75, MGRN, 2, 148.75]
])

# Round 3
app.db.addPlayoffGamesForYearAndRound(2022, 3, [
    # ...
])

In [23]:
app.db.addRecordsFromGames()

### Add Active Years

In [24]:
app.db.addActiveYears()

### Add Top Three Finishes

In [25]:
app.db.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()

app.db.addTopThreeFinishesForYear(2020, [
    [MGR1, 1],
    [MGR2, 2],
    [MGRM, 3]
])

app.db.addTopThreeFinishesForYear(2021, [
    [MGR2, 1],
    [MGRM, 2],
    [MGRN, 3]
])

app.db.addTopThreeFinishesForYear(2022, [
    [MGRM, 1],
    [MGRN, 2],
    [MGR1, 3]
])

## Run

---

Each of the following sections will display some table, chart, or graph to visualize NJFL data. Please see the descriptions for each section and run individual cells to see the output.

### League Managers (2011 - 2022)

Shows a table with all the managers who have participated in the NJFL since it's inception.

In [26]:
app.displayLeagueManagers2()

Name,2020,2021,2022
Manager 1 Name,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Manager 2 Name,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Manager N Name,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Manager N-1 Name,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩


### Manager Records (2011 - 2022)

Shows a table of all league managers records since the inception of the NJFL. Sort values using one of the following values:
- `sort_by.name`, `sort_by.wins`, `sort_by.losses`, `sort_by.ties`, `sort_by.win_pct`

In [27]:
app.displayManagerRecords(sort_by.win_pct)

Name,Wins,Losses,Ties,Win Pct
Manager 1 Name,26,15,0,63.415
Manager 2 Name,25,16,0,60.976
Manager N Name,24,17,0,58.537
Manager N-1 Name,20,21,0,48.780


### Manager Record by Year (2011-2022)

Shows a table of a single managers records for every year they participated in the NJFL.

Use manager name from records table above.

In [28]:
manager = "Manager 1 Name"

app.displayRecordsForManager(manager)

Name,Year,Wins,Losses,Ties,PF,PA
Manager 1 Name,2020,7,6,0,-,-
Manager 1 Name,2021,11,3,0,-,-
Manager 1 Name,2022,8,6,0,1553.90,1493.70


### Total Championships (2011-2021)

Shows a table of total championships for league managers since the inception of the NJFL.

In [29]:
app.displayTotalChampionshipsByManager()

Name,Championships
Manager 1 Name,1
Manager 2 Name,1
Manager N-1 Name,1


### Top Three Finishes (2011-2021)

Shows a table of top three finishes for league managers since the inception of the NJFL.

In [30]:
app.displayTopThreeFinishes()

Name,2020,2021,2022,Total
Manager N-1 Name,🥉,🥈,🥇,3
Manager 1 Name,🥇,,🥉,2
Manager 2 Name,🥈,🥇,,2
Manager N Name,,🥉,🥈,2


### Last Place Finishes (2022)

Shows a table of league last place finishes since this application was created.

In [ ]:
# TODO - use games table to calculate regular season record and PF/PA to determine last place finisher

### League Manager PF & PA (2022)

Shows a table of league managers all-time "points for" and "points against" since this application was created.

In [ ]:
# TODO - use games table to calculate all-time PF and PA, display total PA/PF, and average PF/PA

### League Manager PF & PA by Year

In [ ]:
# TODO - modify above function to handle parameter for year

### Average Game Point Differential

In [ ]:
# TODO

### Average PA & PF

In [ ]:
# TODO

### Highest/Lowest Points

In [ ]:
# TODO